In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/cssws2023


Mounted at /content/drive
/content/drive/MyDrive/cssws2023


In [2]:
%ls

data/                         mecab-ipadic-neologd/  Todai_pandas配布用.ipynb
kokkai_data.ipynb             model/                 Todai_Python_Basics.ipynb
kokkai_simple_analysis.ipynb  sample.csv             Todai_Python_Basics配布用.ipynb
kokkai_tokenize.ipynb         sample.pickle          問題（回答付き）.ipynb
kokkai_topic.ipynb            Todai_pandas.ipynb


## テキスト分析のための前処理
- テキストを量的に処理するためには、言語についてのモデルを作る必要がある。
- もっとも単純なモデルとしてbag of wordsモデルがある。これは文書を単語の集まり（bag of words）としてだけ考え、単語の順番を無視し、出現頻度だけ数えるモデルである。
- モデルとしては、文書をベクトルとし、ベクトルの構成要素はコーパスに出現するすべての単語（ボキャブラリ）であり、当該文書での出現数を値とする。（本セミナーの範囲内では明示的に扱わないが、トピックモデルや単語埋め込みモデル、様々な機械学習で用いられる。なお、chatGPTのような大規模言語モデルは語の順番も考慮するモデルに基づいている）。

<img src="https://raw.githubusercontent.com/berutaki/github.io/main/3-1.png" width="800">


### 形態素解析
- 日本語は膠着語なので語の間の区切りはマークされていない。そのためまず、意味を持つ表現要素の最小単位としての形態素を同定して、区切る必要がある。


<img src="https://raw.githubusercontent.com/berutaki/github.io/main/3-2.png" width="500">

- 形態素解析自体も機械学習によって行われるが、PythonではMecabというソフトウェア（モジュール）を用いて実行できる。


### ストップワード
- 一般的すぎて、文書の特徴付けに貢献しない語
  - 助詞や助動詞のような機能語（です、ます、が、は、でetc.）
  - その他、多くの文書で登場する語（こと、ものetc.）
- ストップワードのリスト
  -   日本語の場合、確立されたリストがあまりない。今回は私が手動で作成したリストを使う。
 https://raw.githubusercontent.com/berutaki/github.io/main/kokkai_stoplist.txt

### 準備
- 必要なライブラリとデータを読み込みます。

In [3]:
import pandas as pd
import numpy as np
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [4]:
dat = pd.read_csv('data/kokkai_corona.csv')
#出席者の名前を記録しているだけの発言を除く
dat = dat.query('speaker != "会議録情報" ')

In [5]:
dat.head()

,speechID,issueID,imageKind,searchObject,session,nameOfHouse,nameOfMeeting,issue,date,closing,...,speaker,speakerYomi,speakerGroup,speakerPosition,speakerRole,speech,startPage,speechURL,meetingURL,pdfURL
0,121104601X01020230413_038,121104601X01020230413,会議録,38,211,衆議院,総務委員会,第10号,2023-04-13,NaN,...,中司宏,なかつかひろし,日本維新の会,NaN,NaN,○中司委員 考え方はよく分かりました。よろしくお願いいたします。\n そこで、オンラインによ...,0,https://kokkai.ndl.go.jp/txt/121104601X0102023...,https://kokkai.ndl.go.jp/txt/121104601X0102023...,NaN
1,121104601X01020230413_099,121104601X01020230413,会議録,99,211,衆議院,総務委員会,第10号,2023-04-13,NaN,...,おおつき紅葉,おおつきくれは,立憲民主党・無所属,NaN,NaN,○おおつき委員 一人の議員の不祥事による補欠選挙で二億円以上のお金がかかる。これは、ちゃんと...,0,https://kokkai.ndl.go.jp/txt/121104601X0102023...,https://kokkai.ndl.go.jp/txt/121104601X0102023...,NaN
2,121104601X01020230413_119,121104601X01020230413,会議録,119,211,衆議院,総務委員会,第10号,2023-04-13,NaN,...,西岡秀子,にしおかひでこ,国民民主党・無所属クラブ,NaN,NaN,○西岡委員 今の御答弁の中でございましたけれども、各地域、選挙管理委員会においては様々なお取...,0,https://kokkai.ndl.go.jp/txt/121104601X0102023...,https://kokkai.ndl.go.jp/txt/121104601X0102023...,NaN
3,121104601X01020230413_123,121104601X01020230413,会議録,123,211,衆議院,総務委員会,第10号,2023-04-13,NaN,...,西岡秀子,にしおかひでこ,国民民主党・無所属クラブ,NaN,NaN,○西岡委員 続きまして、更なる質問になりますけれども、地方議会におけるオンライン開催への取組...,0,https://kokkai.ndl.go.jp/txt/121104601X0102023...,https://kokkai.ndl.go.jp/txt/121104601X0102023...,NaN
4,121104601X01020230413_129,121104601X01020230413,会議録,129,211,衆議院,総務委員会,第10号,2023-04-13,NaN,...,西岡秀子,にしおかひでこ,国民民主党・無所属クラブ,NaN,NaN,○西岡委員 引き続きの待遇改善、処遇改善に是非御尽力いただきたいと思います。\n 最後の質問...,0,https://kokkai.ndl.go.jp/txt/121104601X0102023...,https://kokkai.ndl.go.jp/txt/121104601X0102023...,NaN


### 分かち書き文書の作成
####MeCabとNEologdのインストール
- まずはMecabとNEologdをインストールします（国会会議録データは場合によってはデフォルトの辞書の方がよいかもしれません）インストール法は呪文と考えてください。インストール中に、「Y/n」を聞かれるので、その文言の付近をクリックして、「Y+Enter」としてください。


In [6]:
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール
!apt-get install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl xz-utils make python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
make is already the newest version (4.3-4.1build1).
make set to manually installed.
xz-utils is already the newest version (5.2.5-2ubuntu1).
xz-utils set to manually installed.
curl is already the newest version (7.81.0-1ubuntu1.14).
file is already the newest version (1:5.41-3ubuntu0.1).
git is already the newest version (1:2.34.1-1ubuntu1.10).
The following additional packages will be installed:
  libmecab2 mecab-ipadic mecab-utils
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-utils
0 upgraded, 6 newly installed, 0 to remove and 19 not upgraded.
Need to get 7,367 kB of archives.
After this operation, 59.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libmecab2 amd64 0.996-14build9 [199 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libmecab-dev amd64 0.996-1

In [7]:
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd


In [8]:
!mecab-config --dicdir

/usr/lib/x86_64-linux-gnu/mecab/dic


In [ ]:
# 形態素分析ライブラリーMeCab のインストール
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30 swig4.0
Suggested packages:
  apt-xapian-index aptitude-doc-en | aptitude-doc debtags tasksel libcwidget-dev xapian-tools
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30 swig swig4.0
0 upgraded, 7 newly installed, 0 to remove and 18 not upgraded.
Need to get 4,954 kB of archives.
After this operation, 22.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aptitude-common all 0.8.13-3ubuntu1 [1,719 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsigc++-2.0-0v5 amd64 2.10.4-2ubuntu3 [12.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcwidget4 amd64 0.5.18-5build1 [306 kB]
Get:

In [ ]:
#シンボリックリンク
!ln -f -n -s  /etc/mecabrc /usr/local/etc/mecabrc

In [10]:
#!!! 以下の辞書(mecab-ipadic-NEologd)のインストールで失敗する場合
# fatal: destination path ' ' already exists and is not an empty directory.
#  /content/drive/MyDrive/cssws2023からはgithubからcloneできないのでいったんルートディレクトリに行く。
%cd /

/


In [11]:
#辞書(mecab-ipadic-NEologd)のインストール
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!./mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a -y

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Receiving objects: 100% (75/75), 58.09 MiB | 35.30 MiB/s, done.
Resolving deltas: 100% (5/5), done.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-

In [12]:
%cd /content/drive/MyDrive/cssws2023


/content/drive/MyDrive/cssws2023


In [13]:
#辞書の場所の確認
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"
#/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd

/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd


#### MeCabによる分かち書きの実施
- 名前（空白'\u3000'の前にある）や改行、（拍手）を削除します。
- MeCabで形態素解析をする関数を作成します。
- またストップワードのリストを用意して、形態素解析の際に削除します。
- テキストデータに形態素解析を適用して、分かち書き文書を作成します。
- 最後にdata直下に保存します。

In [14]:
def speech_cleaner(speech):
    item = ''.join(speech.split('\u3000')[1:]).replace('\n','').replace('（拍手）','')
    return item

In [15]:
dat['speech'] = dat['speech'].apply(speech_cleaner)

In [16]:
import MeCab as mc
def tokenize(item):
    m= mc.Tagger("-Ochasen  -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
    n=m.parseToNode(item)
    bow = []
    while n:
        if n.feature.split(',')[0] in ["名詞"]:
            word=n.feature.split(',')[6]#基本形をとる
            if word not in stoplist:
                bow.append(word)#ストップリストになければbowに追加
        n = n.next
    return bow

In [17]:
import urllib.request
url = "https://raw.githubusercontent.com/berutaki/github.io/main/kokkai_stoplist.txt"
with urllib.request.urlopen(url) as f:
  data = f.read().decode('utf-8')
stoplist = data.split('\n')

In [ ]:
stoplist

In [ ]:
bow = dat['speech'].apply(tokenize)

In [ ]:
bow

0        [オンライン, 議会, 参画, 多様, 人材, 確保, 重要, テーマ, 地方公共団体, デ...
1        [一人, 不祥事, 補欠選挙, 2億円, お金, 有権者, 理解, 補欠選挙, 物価高, 選...
2        [地域, 選挙管理委員会, 様々, 取組, 取組, 財政, 質問, 3年, 新型コロナウイル...
3        [更, 質問, 地方議会, オンライン, 開催, 取組, 今年2月, 総務省, 本会議, 質...
4        [待遇改善, 処遇, 改善, 是非, 尽力, 質問, 地方公務員, 定例, 業務, コロナ,...
                               ...                        
39024    [公明党, 斉藤鉄夫, 公明党, 代表, 施政方針演説, 政府四演説, 総理, 関係, 質問...
39025    [斉藤鉄夫, 被災地, 復旧, 復興, 国土, 強靱, 推進, 台風, 甚大, 被害, 政府...
39026    [日本維新の会, 馬場伸幸, 質問, 中国, 発生, 新型コロナウイルス, 肺炎, 感染拡大...
39027    [馬場伸幸, 冒頭, 新型コロナウイルス, 感染拡大, 防止, 政府, 一丸, 全力, 公文...
39028    [情報, 収集, 情報, ここに, 不審船, 情報, 提供, 寄港, 海域, 安全, 情報,...
Name: speech, Length: 38657, dtype: object

In [ ]:
import pickle
with open('data/bow.pickle','wb') as f:
    pickle.dump(bow, f)

### Mecabについて詳しく。

- Mecabでは辞書が選べます。辞書によって語の区切り方が変わってきます。
- NEologdは「恋ダンス」のような固有表現にも対応し、ソーシャルメディアなどのテキストを分かち書きするためにはこちらの方が適切です。


In [19]:
m = mc.Tagger()

In [22]:
sample_txt = "彼女はペンパイナッポーアッポーペンと恋ダンスを踊った。"
print(m.parse(sample_txt))

彼女	名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
ペンパイナッポーアッポーペン	名詞,一般,*,*,*,*,*
と	助詞,並立助詞,*,*,*,*,と,ト,ト
恋	名詞,一般,*,*,*,*,恋,コイ,コイ
ダンス	名詞,サ変接続,*,*,*,*,ダンス,ダンス,ダンス
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
踊っ	動詞,自立,*,*,五段・ラ行,連用タ接続,踊る,オドッ,オドッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。
EOS



In [23]:
path = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
m = mc.Tagger(path)
print("Mecab ipadic NEologd:\n",m.parse(sample_txt))


Mecab ipadic NEologd:
 彼女	名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
ペンパイナッポーアッポーペン	名詞,固有名詞,一般,*,*,*,Pen-Pineapple-Apple-Pen,ペンパイナッポーアッポーペン,ペンパイナッポーアッポーペン
と	助詞,並立助詞,*,*,*,*,と,ト,ト
恋ダンス	名詞,固有名詞,一般,*,*,*,恋ダンス,コイダンス,コイダンス
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
踊っ	動詞,自立,*,*,五段・ラ行,連用タ接続,踊る,オドッ,オドッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。
EOS



In [ ]:
m = mc.Tagger()
sample_txt = "彼女はペンパイナッポーアッポーペンと恋ダンスを踊った。"
print("Mecab:\n", m.parse(sample_txt))

path = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
m = mc.Tagger(path)
print("Mecab ipadic NEologd:\n",m.parse(sample_txt))



Mecab:
 彼女	名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
ペンパイナッポーアッポーペン	名詞,一般,*,*,*,*,*
と	助詞,並立助詞,*,*,*,*,と,ト,ト
恋	名詞,一般,*,*,*,*,恋,コイ,コイ
ダンス	名詞,サ変接続,*,*,*,*,ダンス,ダンス,ダンス
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
踊っ	動詞,自立,*,*,五段・ラ行,連用タ接続,踊る,オドッ,オドッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。
EOS

Mecab ipadic NEologd:
 彼女	名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
ペンパイナッポーアッポーペン	名詞,固有名詞,一般,*,*,*,Pen-Pineapple-Apple-Pen,ペンパイナッポーアッポーペン,ペンパイナッポーアッポーペン
と	助詞,並立助詞,*,*,*,*,と,ト,ト
恋ダンス	名詞,固有名詞,一般,*,*,*,恋ダンス,コイダンス,コイダンス
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
踊っ	動詞,自立,*,*,五段・ラ行,連用タ接続,踊る,オドッ,オドッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。
EOS



### MeCabのオブジェクトの扱い方
- まず辞書を指定して、タガーを生成します。
- parseToNode()メソッドは、形態素（ノード）ごとに出力をしていきます。
- 最初は、BOS/EOS(文頭／文末)を示す符号となります。
- .nextで次のノードに進みます。




In [ ]:
path = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
m = mc.Tagger(path)
print(m.parse(sample_txt))


彼女	名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
ペンパイナッポーアッポーペン	名詞,固有名詞,一般,*,*,*,Pen-Pineapple-Apple-Pen,ペンパイナッポーアッポーペン,ペンパイナッポーアッポーペン
と	助詞,並立助詞,*,*,*,*,と,ト,ト
恋ダンス	名詞,固有名詞,一般,*,*,*,恋ダンス,コイダンス,コイダンス
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
踊っ	動詞,自立,*,*,五段・ラ行,連用タ接続,踊る,オドッ,オドッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。
EOS



In [24]:
n=m.parseToNode(sample_txt)
print(n.feature)


BOS/EOS,*,*,*,*,*,*,*,*


In [25]:
n = n.next
print(n.feature)



名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ


In [26]:
n = n.next
print(n.feature)



助詞,係助詞,*,*,*,*,は,ハ,ワ


- ところでfeatureは形態素の特徴を表す要素を格納しています。
  - o番目が品詞、6番目が基本形（例えば、「踊っ」の基本形は「踊る」）であり、今回はこの2つを用います。
- 文面に表れたとおりの形を表層形（例えば、「踊っ」）といい、surfaceで取得できます。


In [28]:
n.feature.split(',')[0]

['助詞', '係助詞', '*', '*', '*', '*', 'は', 'ハ', 'ワ']

In [29]:
n.feature.split(',')[0]

'助詞'

In [30]:
n.feature.split(',')[6]

'は'

In [31]:
print("品詞: "+n.feature.split(',')[0])
print("基本形: "+n.feature.split(',')[6])
print("表層形:" +n.surface)


品詞: 助詞
基本形: は
表層形:は


- 文章を解析するためには、whileループを使います。
  - さらに品詞を指定して特定の品詞の基本形だけを取り出すなどという操作が可能です。

In [32]:
n=m.parseToNode(sample_txt)
while n:
  if n.feature.split(',')[0] in ["名詞","形容詞","動詞"]:
    print(n.feature.split(',')[6])#基本形のみprint
  n = n.next


彼女
Pen-Pineapple-Apple-Pen
恋ダンス
踊る


### 課題
- 自分で収集したデータを形態素解析して、保存しておきましょう。